In [1]:
from google.colab import drive
drive.mount('/content/drive')

rawdata_path = '/content/drive/MyDrive/my_project/my_project1/rawdata'
data_path = '/content/drive/MyDrive/my_project/my_project1/data'

# rawdata_path = 'rawdata/'
# data_path = 'data/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np

import re
import os

import geopandas as gpd

# 제출용 코드에서 깔끔하게 보이게,..
import warnings
warnings.filterwarnings("ignore")

In [3]:
file_list = os.listdir(f'{rawdata_path}/서울열린데이터광장')
file_list

['사업체현황_종사자수.csv',
 '행정구역(동별).csv',
 '금융기관.csv',
 '유통업체현황.csv',
 '주택종류.csv',
 '대학.csv']

In [4]:
### 종사자수 + 사업체수

In [5]:
company = pd.read_csv(f'{rawdata_path}/서울열린데이터광장/사업체현황_종사자수.csv', header = 1)
company.head()

,동별(1),동별(2),동별(3),합계,합계.1,농업 임업 및 어업,광업,제조업,전기 가스 증기 및 공기조절 공급업,수도 하수 및 폐기물 처리 원료 재생업,...,정보통신업,금융 및 보험업,부동산업,전문 과학 및 기술 서비스업,사업시설 관리 사업 지원 및 임대 서비스업,공공행정 국방 및 사회보장 행정,교육 서비스업,보건업 및 사회복지 서비스업,예술 스포츠 및 여가관련 서비스업,협회 및 단체 수리 및 기타 개인 서비스업
0,동별(1),동별(2),동별(3),사업체수,총종사자수,소계,소계,소계,소계,소계,...,소계,소계,소계,소계,소계,소계,소계,소계,소계,소계
1,합계,소계,소계,1180025,5795425,621,150,261088,6929,10750,...,518475,296873,196379,605708,488351,160530,365167,470395,90030,185909
2,합계,종로구,소계,47595,278403,21,91,12396,685,119,...,17439,23160,9791,29469,19211,15905,13361,21389,5845,8053
3,합계,종로구,사직동,4994,56046,17,1,393,27,15,...,3301,7942,3524,9216,8585,7800,812,1556,1579,923
4,합계,종로구,삼청동,970,5118,-,-,124,-,-,...,167,64,145,521,139,1237,273,53,233,262


In [6]:
company = company[(company['동별(3)'] != '동별(3)') & (company['동별(3)'] != '소계')]
company.head()

,동별(1),동별(2),동별(3),합계,합계.1,농업 임업 및 어업,광업,제조업,전기 가스 증기 및 공기조절 공급업,수도 하수 및 폐기물 처리 원료 재생업,...,정보통신업,금융 및 보험업,부동산업,전문 과학 및 기술 서비스업,사업시설 관리 사업 지원 및 임대 서비스업,공공행정 국방 및 사회보장 행정,교육 서비스업,보건업 및 사회복지 서비스업,예술 스포츠 및 여가관련 서비스업,협회 및 단체 수리 및 기타 개인 서비스업
3,합계,종로구,사직동,4994,56046,17,1,393,27,15,...,3301,7942,3524,9216,8585,7800,812,1556,1579,923
4,합계,종로구,삼청동,970,5118,-,-,124,-,-,...,167,64,145,521,139,1237,273,53,233,262
5,합계,종로구,부암동,908,4140,-,-,89,-,-,...,145,13,75,183,52,60,1247,210,72,998
6,합계,종로구,평창동,1286,4294,-,6,164,-,1,...,184,63,199,246,153,105,664,524,202,237
7,합계,종로구,무악동,837,1987,-,-,8,-,-,...,23,23,103,27,20,20,155,549,26,224


In [7]:
company.drop(columns = '동별(1)', inplace =True)
selected_colunms =  ['시군구', '읍면동', '사업체수_2022', '종사자수_2022'] +company.columns[4:].tolist()
company = company.replace('-', 0)
company.columns = selected_colunms
company.reset_index(drop = True, inplace = True)
company.head()

,시군구,읍면동,사업체수_2022,종사자수_2022,농업 임업 및 어업,광업,제조업,전기 가스 증기 및 공기조절 공급업,수도 하수 및 폐기물 처리 원료 재생업,건설업,...,정보통신업,금융 및 보험업,부동산업,전문 과학 및 기술 서비스업,사업시설 관리 사업 지원 및 임대 서비스업,공공행정 국방 및 사회보장 행정,교육 서비스업,보건업 및 사회복지 서비스업,예술 스포츠 및 여가관련 서비스업,협회 및 단체 수리 및 기타 개인 서비스업
0,종로구,사직동,4994,56046,17,1,393,27,15,557,...,3301,7942,3524,9216,8585,7800,812,1556,1579,923
1,종로구,삼청동,970,5118,0,0,124,0,0,63,...,167,64,145,521,139,1237,273,53,233,262
2,종로구,부암동,908,4140,0,0,89,0,0,63,...,145,13,75,183,52,60,1247,210,72,998
3,종로구,평창동,1286,4294,0,6,164,0,1,161,...,184,63,199,246,153,105,664,524,202,237
4,종로구,무악동,837,1987,0,0,8,0,0,59,...,23,23,103,27,20,20,155,549,26,224


In [8]:
company['읍면동'] = company['읍면동'].apply(lambda x: re.sub(r'\·', '', x))
company['읍면동'] = company['읍면동'].apply(lambda x: re.sub(r'\.', '', x))
company[company['읍면동'].str.contains('\.') | company['읍면동'].str.contains('\·') ]

,시군구,읍면동,사업체수_2022,종사자수_2022,농업 임업 및 어업,광업,제조업,전기 가스 증기 및 공기조절 공급업,수도 하수 및 폐기물 처리 원료 재생업,건설업,...,정보통신업,금융 및 보험업,부동산업,전문 과학 및 기술 서비스업,사업시설 관리 사업 지원 및 임대 서비스업,공공행정 국방 및 사회보장 행정,교육 서비스업,보건업 및 사회복지 서비스업,예술 스포츠 및 여가관련 서비스업,협회 및 단체 수리 및 기타 개인 서비스업


In [9]:
company['구동'] = company['시군구'] + ' ' +company['읍면동']
company.drop(columns = ['시군구', '읍면동'], inplace = True)
company.head()

,사업체수_2022,종사자수_2022,농업 임업 및 어업,광업,제조업,전기 가스 증기 및 공기조절 공급업,수도 하수 및 폐기물 처리 원료 재생업,건설업,도매 및 소매업,운수 및 창고업,...,금융 및 보험업,부동산업,전문 과학 및 기술 서비스업,사업시설 관리 사업 지원 및 임대 서비스업,공공행정 국방 및 사회보장 행정,교육 서비스업,보건업 및 사회복지 서비스업,예술 스포츠 및 여가관련 서비스업,협회 및 단체 수리 및 기타 개인 서비스업,구동
0,4994,56046,17,1,393,27,15,557,3022,2979,...,7942,3524,9216,8585,7800,812,1556,1579,923,종로구 사직동
1,970,5118,0,0,124,0,0,63,853,76,...,64,145,521,139,1237,273,53,233,262,종로구 삼청동
2,908,4140,0,0,89,0,0,63,535,38,...,13,75,183,52,60,1247,210,72,998,종로구 부암동
3,1286,4294,0,6,164,0,1,161,804,136,...,63,199,246,153,105,664,524,202,237,종로구 평창동
4,837,1987,0,0,8,0,0,59,152,483,...,23,103,27,20,20,155,549,26,224,종로구 무악동


In [10]:
file_list[1]

'행정구역(동별).csv'

## 행정구역 면적

In [11]:
area = pd.read_csv(f'{rawdata_path}/서울열린데이터광장/행정구역(동별).csv', header = 2)
area.head()

,동별(1),동별(2),동별(3),면적 (k㎡)
0,서울시,소계,소계,605.21
1,서울시,종로구,소계,23.91
2,서울시,종로구,사직동,1.23
3,서울시,종로구,삼청동,1.49
4,서울시,종로구,부암동,2.27


In [12]:
area = area[(area['동별(3)'] != '동별(3)') & (area['동별(3)'] != '소계')]
area.head()



,동별(1),동별(2),동별(3),면적 (k㎡)
2,서울시,종로구,사직동,1.23
3,서울시,종로구,삼청동,1.49
4,서울시,종로구,부암동,2.27
5,서울시,종로구,평창동,8.87
6,서울시,종로구,무악동,0.36


In [13]:
area.drop(columns = '동별(1)', inplace =True)
selected_colunms =  ['시군구', '읍면동', '면적']
area.columns = selected_colunms
area.reset_index(drop = True, inplace = True)
area.head()

,시군구,읍면동,면적
0,종로구,사직동,1.23
1,종로구,삼청동,1.49
2,종로구,부암동,2.27
3,종로구,평창동,8.87
4,종로구,무악동,0.36


In [14]:
area['읍면동'] = area['읍면동'].apply(lambda x: re.sub(r'\·', '', x))
area['읍면동'] = area['읍면동'].apply(lambda x: re.sub(r'\.', '', x))


area['구동'] = area['시군구'] + ' ' +area['읍면동']
area.drop(columns = ['시군구', '읍면동'], inplace = True)
area.head()

,면적,구동
0,1.23,종로구 사직동
1,1.49,종로구 삼청동
2,2.27,종로구 부암동
3,8.87,종로구 평창동
4,0.36,종로구 무악동


In [15]:
file_list[2]

'금융기관.csv'

In [16]:
## 금융기관

In [17]:
finance = pd.read_csv(f'{rawdata_path}/서울열린데이터광장/금융기관.csv', header = 0)
finance.head()

,동별(1),동별(2),동별(3),2022,2023
0,동별(1),동별(2),동별(3),금융기관,금융기관
1,합계,종로구,사직동,23,21
2,합계,종로구,삼청동,3,3
3,합계,종로구,부암동,1,1
4,합계,종로구,평창동,2,2


In [18]:
finance.columns = ['합계', '시군구', '읍면동', '금융기관_2022', '금융기관_2023']
finance.drop(columns = '합계', inplace =True)
finance = finance[finance['읍면동'] != '소계']
finance = finance[finance['읍면동'] != '동별(3)']
finance.reset_index(drop = True, inplace = True)
finance.head()

,시군구,읍면동,금융기관_2022,금융기관_2023
0,종로구,사직동,23,21
1,종로구,삼청동,3,3
2,종로구,부암동,1,1
3,종로구,평창동,2,2
4,종로구,무악동,1,1


In [19]:
finance['읍면동'] = finance['읍면동'].apply(lambda x: re.sub(r'\·', '', x))
finance['읍면동'] = finance['읍면동'].apply(lambda x: re.sub(r'\.', '', x))

finance['구동'] = finance['시군구'] + ' ' +finance['읍면동']
finance.drop(columns = ['시군구', '읍면동'], inplace = True)
finance.head()

,금융기관_2022,금융기관_2023,구동
0,23,21,종로구 사직동
1,3,3,종로구 삼청동
2,1,1,종로구 부암동
3,2,2,종로구 평창동
4,1,1,종로구 무악동


In [20]:
finance

,금융기관_2022,금융기관_2023,구동
0,23,21,종로구 사직동
1,3,3,종로구 삼청동
2,1,1,종로구 부암동
3,2,2,종로구 평창동
4,1,1,종로구 무악동
...,...,...,...
74,7,7,광진구 광장동
75,4,4,광진구 자양1동
76,1,1,광진구 자양2동
77,5,5,광진구 자양3동


In [21]:
file_list[3]

'유통업체현황.csv'

## 유통업체

In [22]:
mart = pd.read_csv(f'{rawdata_path}/서울열린데이터광장/유통업체현황.csv', header = 1)
mart.head()

,동별(1),동별(2),동별(3),합계,대형마트,백화점,전문점,쇼핑센터,복합쇼핑몰,그밖의 대규모점포
0,동별(1),동별(2),동별(3),소계,소계,소계,소계,소계,소계,소계
1,동별(1),동별(2),동별(3),개소 (개소),개소 (개소),개소 (개소),개소 (개소),개소 (개소),개소 (개소),개소 (개소)
2,서울시,소계,소계,444,55,31,24,41,8,285
3,서울시,종로구,소계,13,-,-,2,-,-,11
4,서울시,종로구,사직동,-,-,-,-,-,-,-


In [23]:
mart = mart[(mart['동별(3)'] != '동별(3)') & (mart['동별(3)'] != '소계')]
mart.head()

,동별(1),동별(2),동별(3),합계,대형마트,백화점,전문점,쇼핑센터,복합쇼핑몰,그밖의 대규모점포
4,서울시,종로구,사직동,-,-,-,-,-,-,-
5,서울시,종로구,삼청동,-,-,-,-,-,-,-
6,서울시,종로구,부암동,-,-,-,-,-,-,-
7,서울시,종로구,평창동,-,-,-,-,-,-,-
8,서울시,종로구,무악동,-,-,-,-,-,-,-


In [24]:
mart.drop(columns = '동별(1)', inplace =True)
selected_colunms =  ['시군구', '읍면동', '유통업체수_2022'] +mart.columns[3:].tolist()
mart.columns = selected_colunms
mart.reset_index(drop = True, inplace = True)

mart = mart.replace('-', 0)

mart.head()

,시군구,읍면동,유통업체수_2022,대형마트,백화점,전문점,쇼핑센터,복합쇼핑몰,그밖의 대규모점포
0,종로구,사직동,0,0,0,0,0,0,0
1,종로구,삼청동,0,0,0,0,0,0,0
2,종로구,부암동,0,0,0,0,0,0,0
3,종로구,평창동,0,0,0,0,0,0,0
4,종로구,무악동,0,0,0,0,0,0,0


In [25]:
mart['읍면동'] = mart['읍면동'].apply(lambda x: re.sub(r'\·', '', x))
mart['읍면동'] = mart['읍면동'].apply(lambda x: re.sub(r'\.', '', x))
mart[mart['읍면동'].str.contains('\.') | mart['읍면동'].str.contains('\·') ]

,시군구,읍면동,유통업체수_2022,대형마트,백화점,전문점,쇼핑센터,복합쇼핑몰,그밖의 대규모점포


In [26]:
mart['유통업체수_대형마트외'] = mart['유통업체수_2022'].astype(int) - mart['대형마트'].astype(int)

In [27]:
mart['구동'] = mart['시군구'] + ' ' +mart['읍면동']
mart.drop(columns = ['시군구', '읍면동'], inplace = True)
mart.head()

,유통업체수_2022,대형마트,백화점,전문점,쇼핑센터,복합쇼핑몰,그밖의 대규모점포,유통업체수_대형마트외,구동
0,0,0,0,0,0,0,0,0,종로구 사직동
1,0,0,0,0,0,0,0,0,종로구 삼청동
2,0,0,0,0,0,0,0,0,종로구 부암동
3,0,0,0,0,0,0,0,0,종로구 평창동
4,0,0,0,0,0,0,0,0,종로구 무악동


In [28]:
file_list[4]

'주택종류.csv'

# 주택종류

In [29]:
house = pd.read_csv(f'{rawdata_path}/서울열린데이터광장/주택종류.csv', header = 2)
house.head()

,동별(1),동별(2),동별(3),소계,단독주택,단독주택.1,단독주택.2,단독주택.3,아파트,연립주택,다세대주택,비거주용건물내주택
0,동별(1),동별(2),동별(3),소계,소계,일반단독주택,다가구주택,영업겸용주택,소계,소계,소계,소계
1,서울시,소계,소계,3015371,307075,64911,190399,51765,1772670,110562,796066,28998
2,서울시,종로구,소계,46859,12259,6572,4070,1617,15688,5460,12540,912
3,서울시,종로구,사직동,2778,650,467,108,75,1591,78,411,48
4,서울시,종로구,삼청동,730,644,414,134,96,X,19,37,30


In [30]:
house = house[(house['동별(3)'] != '동별(3)') & (house['동별(3)'] != '소계')]
house.head()

,동별(1),동별(2),동별(3),소계,단독주택,단독주택.1,단독주택.2,단독주택.3,아파트,연립주택,다세대주택,비거주용건물내주택
3,서울시,종로구,사직동,2778,650,467,108,75,1591,78,411,48
4,서울시,종로구,삼청동,730,644,414,134,96,X,19,37,30
5,서울시,종로구,부암동,3070,1033,750,214,69,121,903,976,37
6,서울시,종로구,평창동,5958,1553,1088,314,151,1009,2186,1139,71
7,서울시,종로구,무악동,2860,48,14,9,25,2703,55,46,8


In [31]:
house.drop(columns = '동별(1)', inplace =True)
selected_colunms =  ['시군구', '읍면동', '주택수_2020', '단독주택수','일반단독주택', '다가구주택', '영업겸용주택' ] +house.columns[7:].tolist()
house.columns = selected_colunms
house.reset_index(drop = True, inplace = True)

house = house.replace('-', 0)
house = house.replace('X', 0)

house.head()

,시군구,읍면동,주택수_2020,단독주택수,일반단독주택,다가구주택,영업겸용주택,아파트,연립주택,다세대주택,비거주용건물내주택
0,종로구,사직동,2778,650,467,108,75,1591,78,411,48
1,종로구,삼청동,730,644,414,134,96,0,19,37,30
2,종로구,부암동,3070,1033,750,214,69,121,903,976,37
3,종로구,평창동,5958,1553,1088,314,151,1009,2186,1139,71
4,종로구,무악동,2860,48,14,9,25,2703,55,46,8


In [32]:
house['읍면동'] = house['읍면동'].apply(lambda x: re.sub(r'\·', '', x))
house['읍면동'] = house['읍면동'].apply(lambda x: re.sub(r'\.', '', x))
house[house['읍면동'].str.contains('\.') | house['읍면동'].str.contains('\·') ]

,시군구,읍면동,주택수_2020,단독주택수,일반단독주택,다가구주택,영업겸용주택,아파트,연립주택,다세대주택,비거주용건물내주택


In [33]:
house['구동'] = house['시군구'] + ' ' +house['읍면동']
house.drop(columns = ['시군구', '읍면동'], inplace = True)
house.head()

,주택수_2020,단독주택수,일반단독주택,다가구주택,영업겸용주택,아파트,연립주택,다세대주택,비거주용건물내주택,구동
0,2778,650,467,108,75,1591,78,411,48,종로구 사직동
1,730,644,414,134,96,0,19,37,30,종로구 삼청동
2,3070,1033,750,214,69,121,903,976,37,종로구 부암동
3,5958,1553,1088,314,151,1009,2186,1139,71,종로구 평창동
4,2860,48,14,9,25,2703,55,46,8,종로구 무악동


In [34]:
## 강남구 일원2동 명칭 변경 -> 개포3동

In [35]:
house.loc[373, '구동'] = '강남구 개포3동'

In [36]:
file_list[5]

'대학.csv'

In [37]:
univ_raw = pd.read_csv(f'{rawdata_path}/서울열린데이터광장/대학.csv', header = 0, encoding ='cp949')
univ_raw.head()

,키값,연도,학교종류,학교명,본분교,학교상태,설립,우편번호,주소,도로명주소,행정시,행정구,행정동,전화번호,팩스번호,홈페이지
0,BE_LiST23-0034,2013,일반대학,서울시립대학교,본교,기존,공립,NaN,서울 동대문구 서울시립대로 163 (전농동 90번지),NaN,서울특별시,동대문구,휘경2동,02-6490-6114,02-2210-5576,http://www.uos.ac.kr
1,BE_LiST23-0035,2013,전문대학(3년제),서울여자간호대학교,본교,기존,사립,NaN,서울 서대문구 홍제3동 서울여자간호대학,NaN,서울특별시,서대문구,홍제2동,02-2287-1700,02-395-8018,http://www.snjc.ac.kr
2,BE_LiST23-0036,2013,일반대학,서울여자대학교,본교,기존,사립,NaN,서울특별시 노원구 화랑로 621 서울여자대학교,NaN,서울특별시,노원구,공릉2동,02-970-5114,02-978-7931,http://www.swu.ac.kr
3,BE_LiST23-0037,2013,전문대학(2년제),서일대학교,본교,학교명 변경,사립,NaN,서울 중랑구 서일대학길 22(면목동 49-3) 서일대학교,NaN,서울특별시,중랑구,면목제3.8동,02-490-7300,02-493-2576,http://www.seoil.ac.kr
4,BE_LiST23-0038,2013,일반대학,성공회대학교,본교,기존,사립,NaN,서울 구로구 항동 성공회대학교,NaN,서울특별시,구로구,오류2동,02-2610-4142,02-2610-4248,http://www.skhu.ac.kr


In [38]:
univ_raw = univ_raw[['학교종류', '학교명', '행정구', '행정동']]
univ_raw

,학교종류,학교명,행정구,행정동
0,일반대학,서울시립대학교,동대문구,휘경2동
1,전문대학(3년제),서울여자간호대학교,서대문구,홍제2동
2,일반대학,서울여자대학교,노원구,공릉2동
3,전문대학(2년제),서일대학교,중랑구,면목제3.8동
4,일반대학,성공회대학교,구로구,오류2동
...,...,...,...,...
59,교육대학,서울교육대학교,서초구,서초1동
60,일반대학,서울기독대학교,은평구,신사1동
61,일반대학,서울대학교,관악구,낙성대동
62,사이버대학(대학),서울디지털대학교,마포구,도화동


In [39]:
univ_raw['행정동'] = univ_raw['행정동'].apply(lambda x: re.sub(r'\·', '', x))
univ_raw['행정동'] = univ_raw['행정동'].apply(lambda x: re.sub(r'\.', '', x))

univ_raw['구동'] = univ_raw['행정구'] + ' ' +univ_raw['행정동']
univ_raw.drop(columns = ['행정구', '행정동'], inplace = True)
univ_raw.head()

,학교종류,학교명,구동
0,일반대학,서울시립대학교,동대문구 휘경2동
1,전문대학(3년제),서울여자간호대학교,서대문구 홍제2동
2,일반대학,서울여자대학교,노원구 공릉2동
3,전문대학(2년제),서일대학교,중랑구 면목제38동
4,일반대학,성공회대학교,구로구 오류2동


In [40]:
temp_univ = univ_raw.groupby(['구동', '학교종류'])[['학교명']].count()
temp_univ.reset_index(inplace = True)
temp_univ.head(10)

,구동,학교종류,학교명
0,강남구 논현2동,전공대학,1
1,강북구 미아동,사이버대학(대학),1
2,강서구 우장산동,기능대학,1
3,강서구 화곡본동,일반대학,1
4,관악구 낙성대동,일반대학,1
5,광진구 광장동,일반대학,1
6,광진구 군자동,일반대학,1
7,광진구 화양동,일반대학,1
8,구로구 개봉1동,일반대학,1
9,구로구 고척1동,전문대학(3년제),1


In [41]:
univ = pd.pivot_table(temp_univ, values='학교명', index='구동', columns='학교종류', aggfunc='sum', fill_value=0)
univ.head()

학교종류,각종대학(대학),교육대학,기능대학,방송통신대학,사이버대학(대학),산업대학,일반대학,전공대학,전문대학(2년제),전문대학(3년제)
구동,,,,,,,,,,
강남구 논현2동,0,0,0,0,0,0,0,1,0,0
강북구 미아동,0,0,0,0,1,0,0,0,0,0
강서구 우장산동,0,0,1,0,0,0,0,0,0,0
강서구 화곡본동,0,0,0,0,0,0,1,0,0,0
관악구 낙성대동,0,0,0,0,0,0,1,0,0,0


In [42]:
univ.columns

Index(['각종대학(대학)', '교육대학', '기능대학', '방송통신대학', '사이버대학(대학)', '산업대학', '일반대학',
       '전공대학', '전문대학(2년제)', '전문대학(3년제)'],
      dtype='object', name='학교종류')

In [43]:
univ4 = ['교육대학', '일반대학']
univ_etc = [col for col in univ.columns if col not in univ4]
univ_etc

['각종대학(대학)',
 '기능대학',
 '방송통신대학',
 '사이버대학(대학)',
 '산업대학',
 '전공대학',
 '전문대학(2년제)',
 '전문대학(3년제)']

In [44]:
univ['대학교'] = univ[univ4].sum(axis = 1)
univ['기타대학'] = univ[univ_etc].sum(axis = 1)
univ.reset_index(inplace = True)
univ.head()

학교종류,구동,각종대학(대학),교육대학,기능대학,방송통신대학,사이버대학(대학),산업대학,일반대학,전공대학,전문대학(2년제),전문대학(3년제),대학교,기타대학
0,강남구 논현2동,0,0,0,0,0,0,0,1,0,0,0,1
1,강북구 미아동,0,0,0,0,1,0,0,0,0,0,0,1
2,강서구 우장산동,0,0,1,0,0,0,0,0,0,0,0,1
3,강서구 화곡본동,0,0,0,0,0,0,1,0,0,0,1,0
4,관악구 낙성대동,0,0,0,0,0,0,1,0,0,0,1,0


In [45]:
univ = univ[['구동', '대학교', '기타대학']]
univ.head()

학교종류,구동,대학교,기타대학
0,강남구 논현2동,0,1
1,강북구 미아동,0,1
2,강서구 우장산동,0,1
3,강서구 화곡본동,1,0
4,관악구 낙성대동,1,0


In [46]:
univ

학교종류,구동,대학교,기타대학
0,강남구 논현2동,0,1
1,강북구 미아동,0,1
2,강서구 우장산동,0,1
3,강서구 화곡본동,1,0
4,관악구 낙성대동,1,0
5,광진구 광장동,1,0
6,광진구 군자동,1,0
7,광진구 화양동,1,0
8,구로구 개봉1동,1,0
9,구로구 고척1동,0,1


In [47]:
file_list

['사업체현황_종사자수.csv',
 '행정구역(동별).csv',
 '금융기관.csv',
 '유통업체현황.csv',
 '주택종류.csv',
 '대학.csv']

# df 결합

In [48]:
# df_list = ['company', 'area', 'finance', 'mart', 'house', 'univ']

In [49]:
df_list = ['company', 'area', 'finance', 'mart', 'house', 'univ']
merged_df = pd.DataFrame()

for idx, df_name in enumerate(df_list):
    df = globals()[df_name]
    if idx == 0:
        merged_df = df
    else :
        merged_df = merged_df.merge(df,
                                    how = 'left',
                                    on = '구동')


In [50]:
merged_df.isna().sum()

사업체수_2022                     0
종사자수_2022                     0
농업 임업 및 어업                    0
광업                            0
제조업                           0
전기 가스 증기 및 공기조절 공급업           0
수도 하수 및 폐기물 처리 원료 재생업         0
건설업                           0
도매 및 소매업                      0
운수 및 창고업                      0
숙박 및 음식점업                     0
정보통신업                         0
금융 및 보험업                      0
부동산업                          0
전문 과학 및 기술 서비스업               0
사업시설 관리 사업 지원 및 임대 서비스업       0
공공행정 국방 및 사회보장 행정             0
교육 서비스업                       0
보건업 및 사회복지 서비스업               0
예술 스포츠 및 여가관련 서비스업            0
 협회 및 단체 수리 및 기타 개인 서비스업      0
구동                            0
면적                            0
금융기관_2022                   347
금융기관_2023                   347
유통업체수_2022                    0
대형마트                          0
백화점                           0
전문점                           0
쇼핑센터                          0
복합쇼핑몰                         0
그밖의 대규모점

In [51]:
merged_df.fillna(0, inplace = True)

In [52]:
dong_convert = pd.read_csv(f'{data_path}/region_code.csv')
dong_convert

,ADM_CD,행정동코드,시군구,행정동1,행정동2,key1,key2
0,11010720.0,1111051500,종로구,청운효자동,청운효자동,종로구 청운효자동,종로구 청운효자동
1,11010530.0,1111053000,종로구,사직동,사직동,종로구 사직동,종로구 사직동
2,11010540.0,1111054000,종로구,삼청동,삼청동,종로구 삼청동,종로구 삼청동
3,11010550.0,1111055000,종로구,부암동,부암동,종로구 부암동,종로구 부암동
4,11010560.0,1111056000,종로구,평창동,평창동,종로구 평창동,종로구 평창동
...,...,...,...,...,...,...,...
421,11250660.0,1174065000,강동구,성내2동,성내제2동,강동구 성내2동,강동구 성내제2동
422,11250670.0,1174066000,강동구,성내3동,성내제3동,강동구 성내3동,강동구 성내제3동
423,11250740.0,1174068500,강동구,길동,길동,강동구 길동,강동구 길동
424,11250700.0,1174069000,강동구,둔촌1동,둔촌제1동,강동구 둔촌1동,강동구 둔촌제1동


In [53]:
final = pd.merge(dong_convert.rename(columns = {'key1' : 'key'}),
                 merged_df.rename(columns = {'구동' : 'key'}),
                 how = 'left',
                 on = 'key')

In [54]:
final = final[['ADM_CD'] + final.columns[7:].tolist()]
final.head()

,ADM_CD,사업체수_2022,종사자수_2022,농업 임업 및 어업,광업,제조업,전기 가스 증기 및 공기조절 공급업,수도 하수 및 폐기물 처리 원료 재생업,건설업,도매 및 소매업,...,단독주택수,일반단독주택,다가구주택,영업겸용주택,아파트,연립주택,다세대주택,비거주용건물내주택,대학교,기타대학
0,11010720.0,1469,5433,0,0,145,4,0,148,680,...,1279,834,306,139,260,787,1750,58,0.0,1.0
1,11010530.0,4994,56046,17,1,393,27,15,557,3022,...,650,467,108,75,1591,78,411,48,0.0,0.0
2,11010540.0,970,5118,0,0,124,0,0,63,853,...,644,414,134,96,0,19,37,30,0.0,0.0
3,11010550.0,908,4140,0,0,89,0,0,63,535,...,1033,750,214,69,121,903,976,37,1.0,0.0
4,11010560.0,1286,4294,0,6,164,0,1,161,804,...,1553,1088,314,151,1009,2186,1139,71,0.0,0.0


In [55]:
final.isna().sum()

ADM_CD                      0
사업체수_2022                   0
종사자수_2022                   0
농업 임업 및 어업                  0
광업                          0
제조업                         0
전기 가스 증기 및 공기조절 공급업         0
수도 하수 및 폐기물 처리 원료 재생업       0
건설업                         0
도매 및 소매업                    0
운수 및 창고업                    0
숙박 및 음식점업                   0
정보통신업                       0
금융 및 보험업                    0
부동산업                        0
전문 과학 및 기술 서비스업             0
사업시설 관리 사업 지원 및 임대 서비스업     0
공공행정 국방 및 사회보장 행정           0
교육 서비스업                     0
보건업 및 사회복지 서비스업             0
예술 스포츠 및 여가관련 서비스업          0
 협회 및 단체 수리 및 기타 개인 서비스업    0
면적                          0
금융기관_2022                   0
금융기관_2023                   0
유통업체수_2022                  0
대형마트                        0
백화점                         0
전문점                         0
쇼핑센터                        0
복합쇼핑몰                       0
그밖의 대규모점포                   0
유통업체수_대형마트외                 0
주택수_2020  

In [56]:
final.to_csv(f'{data_path}/final/seoul.csv', index = False)